In [7]:
import os
import sys
import numpy as np
import argparse
from pyrite import _utils as ut
import glob
import matplotlib.pyplot as plt
from gigpy import elevation_models

import scipy.interpolate as si
import scipy.interpolate as intp

from scipy.optimize import leastsq
from scipy.stats.stats import pearsonr

from pykrige import OrdinaryKriging

from mintpy.utils import ptime
import scipy.interpolate as intp
import scipy.integrate as intg
import scipy.spatial as ss

import matlab.engine
#### define dicts for model/residual/initial values ###############
def make3dintp(Delfn,lonlist,latlist,hgt,hgtscale):
    '''Returns a 3D interpolation function that can be used to interpolate using llh coordinates.

    Args:
        * Delfn    (np.array) : Array of delay values.
        * lonlist  (np.array) : Array of station longitudes.
        * latlist  (np.array) : Array of station latitudes.
        * hgt      (np.array) : Array of height levels.
        * hgtscale (np.float) : Height scale factor for interpolator.

    Returns:
        * fnc  (function) : 3D interpolation function.

    .. note::
        We currently use the LinearNDInterpolator from scipy.
        '''
    ##Delfn   = Ddry + Dwet. Delay function.
    ##lonlist = list of lons for stations. / x
    ##latlist = list of lats for stations. / y
    nstn = Delfn.shape[0]
    nhgt = Delfn.shape[1]
    xyz = np.zeros((nstn*nhgt,3))
    Delfn = np.reshape(Delfn,(nstn*nhgt,1))
    print('3D interpolation')
    count = 0
    for m in range(nstn):
        for n in range(nhgt):
            xyz[count,0] = lonlist[m]
            xyz[count,1] = latlist[m]
            xyz[count,2] = hgt[n]/hgtscale     #For same grid spacing as lat/lon
            count += 1

    #xyz[:,2] = xyz[:,2] #+ 1e-30*np.random.rand((nstn*nhgt))/hgtscale #For unique Delaunay    
    del latlist
    del lonlist
    del hgt
    if verbose:
        print('PROGRESS: BUILDING INTERPOLATION FUNCTION')
    fnc = intp.LinearNDInterpolator(xyz,Delfn)

    return fnc

def remove_ramp(lat,lon,data):
    # mod = a*x + b*y + c*x*y
    lat = lat/180*np.pi
    lon = lon/180*np.pi  
    lon = lon*np.cos(lat) # to get isometrics coordinates
    
    p0 = [0.0001,0.0001,0.0001,0.0000001]
    plsq = leastsq(residual_trend,p0,args = (lat,lon,data))
    para = plsq[0]
    data_trend = data - func_trend(lat,lon,para)
    corr, _ = pearsonr(data, func_trend(lat,lon,para))
    return data_trend, para, corr

def func_trend(lat,lon,p):
    a0,b0,c0,d0 = p
    
    return a0 + b0*lat + c0*lon +d0*lat*lon


def func_trend_model(lat,lon,p):
    lat = lat/180*np.pi
    lon = lon/180*np.pi  
    lon = lon*np.cos(lat) # to get isometrics coordinates
    a0,b0,c0,d0 = p
    
    return a0 + b0*lat + c0*lon +d0*lat*lon

def residual_trend(p,lat,lon,y0):
    a0,b0,c0,d0 = p 
    return y0 - func_trend(lat,lon,p)



model_dict = {'linear': elevation_models.linear_elevation_model,
                      'onn': elevation_models.onn_elevation_model,
                      'onn_linear': elevation_models.onn_linear_elevation_model,
                      'exp': elevation_models.exp_elevation_model,
                      'exp_linear': elevation_models.exp_linear_elevation_model}

residual_dict = {'linear': elevation_models.residuals_linear,
                      'onn': elevation_models.residuals_onn,
                      'onn_linear': elevation_models.residuals_onn_linear,
                      'exp': elevation_models.residuals_exp,
                      'exp_linear': elevation_models.residuals_exp_linear}

initial_dict = {'linear': elevation_models.initial_linear,
                      'onn': elevation_models.initial_onn,
                      'onn_linear': elevation_models.initial_onn_linear,
                      'exp': elevation_models.initial_exp,
                      'exp_linear': elevation_models.initial_exp_linear}

para_numb_dict = {'linear': 2,
                  'onn' : 3,
                  'onn_linear':4,
                  'exp':2,
                  'exp_linear':3}

font = {'family': 'serif',
        'color':  'black',
        'weight': 'bold',
        'size': 12,
        }

cdic = ut.initconst()
raw_dir = '/Users/caoy0a/Documents/SCRATCH/AqabaT87S1A/pyrite/ERA5/raw'
date0 = '20160602'
#print(date0)
fname0 = glob.glob(raw_dir + '/ERA*' + date0 + '*')[0]
#print(fname0)
lvls,latlist,lonlist,gph,tmp,vpr = ut.get_ecmwf('ERA5',fname0,cdic, humidity='Q')
lonlist = lonlist - 360.0
lon = lonlist.flatten() 
lat = latlist.flatten() 

#print(lon)
#print(lat)
lonStep = lonlist[0,1] - lonlist[0,0]
latStep = latlist[1,0] - latlist[0,0]

maxlon = max(lonlist[0,:])
minlon = min(lonlist[0,:])

maxlat = max(latlist[:,0])
minlat = min(latlist[:,0])

#print(lonStep)
#print(latStep)

Rescale = 5
lonStep1 = lonStep/Rescale
latStep1 = latStep/Rescale

lonv = np.arange(minlon,maxlon,lonStep1)
latv = np.arange(maxlat,minlat,latStep1)

#print(lonv)
#print(latv)

lonvv,latvv = np.meshgrid(lonv,latv)


# Make a height scale
hgt = np.linspace(cdic['minAltP'], gph.max().round(), cdic['nhgt'])
# Interpolate pressure, temperature and Humidity of hgt
[Pi,Ti,Vi] = ut.intP2H(lvls, hgt, gph, tmp, vpr, cdic)
# Calculate the delays
[DDry,DWet] = ut.PTV2del(Pi,Ti,Vi,hgt,cdic)

print(cdic['minAltP'])
print(cdic['nhgt'])

print(hgt.shape)
print(hgt)

print(lvls.shape)
print(gph.shape)
print(vpr.shape)

-200.0
300
(300,)
[-2.00000000e+02 -3.78026756e+01  1.24394649e+02  2.86591973e+02
  4.48789298e+02  6.10986622e+02  7.73183946e+02  9.35381271e+02
  1.09757860e+03  1.25977592e+03  1.42197324e+03  1.58417057e+03
  1.74636789e+03  1.90856522e+03  2.07076254e+03  2.23295987e+03
  2.39515719e+03  2.55735452e+03  2.71955184e+03  2.88174916e+03
  3.04394649e+03  3.20614381e+03  3.36834114e+03  3.53053846e+03
  3.69273579e+03  3.85493311e+03  4.01713043e+03  4.17932776e+03
  4.34152508e+03  4.50372241e+03  4.66591973e+03  4.82811706e+03
  4.99031438e+03  5.15251171e+03  5.31470903e+03  5.47690635e+03
  5.63910368e+03  5.80130100e+03  5.96349833e+03  6.12569565e+03
  6.28789298e+03  6.45009030e+03  6.61228763e+03  6.77448495e+03
  6.93668227e+03  7.09887960e+03  7.26107692e+03  7.42327425e+03
  7.58547157e+03  7.74766890e+03  7.90986622e+03  8.07206355e+03
  8.23426087e+03  8.39645819e+03  8.55865552e+03  8.72085284e+03
  8.88305017e+03  9.04524749e+03  9.20744482e+03  9.36964214e+03
  9.531